In [47]:
import pandas as pd
import numpy as np
import os

In [48]:
bbdd_folder = 'C:/Users/Marta/Documents/RecommenderSystem/BBDD'

In [49]:
orders_df = pd.read_csv(os.path.join(bbdd_folder, 'orders.csv'))

In [50]:
orders_train_df = pd.read_csv(os.path.join(bbdd_folder, 'order_products__train.csv'))

In [51]:
merged_df = pd.merge(orders_train_df, orders_df, on='order_id', how='left') 

In [52]:
f_df = merged_df.dropna(how='any')

In [53]:
f_df['ratings'] = (f_df['reordered']) / (f_df['days_since_prior_order'])

In [54]:
f_df.replace([np.inf, 0], np.nan).dropna(how='any', inplace=True)

In [55]:
f_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1384617 entries, 0 to 1384616
Data columns (total 11 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   order_id                1384617 non-null  int64  
 1   product_id              1384617 non-null  int64  
 2   add_to_cart_order       1384617 non-null  int64  
 3   reordered               1384617 non-null  int64  
 4   user_id                 1384617 non-null  int64  
 5   eval_set                1384617 non-null  object 
 6   order_number            1384617 non-null  int64  
 7   order_dow               1384617 non-null  int64  
 8   order_hour_of_day       1384617 non-null  int64  
 9   days_since_prior_order  1384617 non-null  float64
 10  ratings                 1381621 non-null  float64
dtypes: float64(2), int64(8), object(1)
memory usage: 126.8+ MB


In [56]:
f_df = f_df.drop(
        ['eval_set', 'add_to_cart_order', 'reordered', 'order_number', 'order_dow', 'order_hour_of_day',
         'days_since_prior_order'],
        1)

In [57]:
not_dup_df = f_df.drop_duplicates(['user_id', 'product_id'], keep='first')

In [87]:
def train_test_val_split(data, test_ratio, val_ratio):
    """
        Splits train and test sets
        :param original_df:
        :return (dict) results:
    """
    seed = np.random.seed(42)
    shuffle_idx = np.random.permutation(len(data))
    test_size = int(len(data) * test_ratio)
    val_size = int(len(data) * val_ratio)
    train_indices = shuffle_idx[:len(data) - val_size - test_size - 1]
    val_indices = shuffle_idx[(len(data)- val_size - test_size):(len(data)-test_size-1)]
    test_indices = shuffle_idx[(len(data)-test_size):]

    train_df = data.iloc[train_indices]
    test_df = data.iloc[test_indices]
    val_df = data.iloc[val_indices]
    return train_df, test_df, val_df
    return shuffle_idx

In [88]:
train_df, test_df, val_df = train_test_val_split(not_dup_df, 0.2, 0.1)